In [11]:
import os
import json
import numpy as np
import pandas as pd
import time, logging
import gensim
from gensim.models import Phrases, KeyedVectors
from string import punctuation
import re
import unicodedata

In [12]:
!ls -lah ../data

total 1136880
drwxr-xr-x  11 rsilvei  ADESI\Domain Users   352B Aug 19 23:08 .
drwxr-xr-x   6 rsilvei  ADESI\Domain Users   192B Aug 14 10:51 ..
drwxr-xr-x   2 rsilvei  ADESI\Domain Users    64B Aug 14 22:25 jogo_gremio_athletico
-rw-r--r--   1 rsilvei  ADESI\Domain Users   100M Aug 14 22:18 tweets_1.txt
-rw-r--r--   1 rsilvei  ADESI\Domain Users    48M Aug 16 14:20 tweets_2_d47c92ce-501e-480a-afd9-f30343d8474f.txt
-rw-r--r--   1 rsilvei  ADESI\Domain Users    29M Aug 14 20:37 tweets_3.txt
-rw-r--r--   1 rsilvei  ADESI\Domain Users   104M Aug 15 09:40 tweets_4b05f867-a2ba-418d-926b-857288867813.txt
-rw-r--r--   1 rsilvei  ADESI\Domain Users    50M Aug 19 10:56 tweets_5b8da1aa-a089-4b42-9c49-64d2cde4dc52.txt
-rw-r--r--   1 rsilvei  ADESI\Domain Users    44M Aug 20 00:15 tweets_84db8ab8-4af8-4945-96ea-10662d9f74ae.txt
-rw-r--r--   1 rsilvei  ADESI\Domain Users   153M Aug 18 20:13 tweets_a3e3c5c4-7181-4168-bfbc-ccd13cf1aed2.txt
-rw-r--r--@  1 rsilvei  ADESI\Domain Users   1.6K Aug 15 10:0

In [36]:
tweet_files = [
    'tweets_1.txt',
    'tweets_2_d47c92ce-501e-480a-afd9-f30343d8474f.txt',
    'tweets_3.txt',
    'tweets_4b05f867-a2ba-418d-926b-857288867813.txt',
    'tweets_5b8da1aa-a089-4b42-9c49-64d2cde4dc52.txt',
    'tweets_84db8ab8-4af8-4945-96ea-10662d9f74ae.txt',
    'tweets_a3e3c5c4-7181-4168-bfbc-ccd13cf1aed2.txt'  
]

In [37]:
tweet_files[0]

'tweets_1.txt'

In [38]:
tweets_data = []
for file in tweet_files:
    tweets_file = open(os.path.join('../data', file), "r")
    for line in tweets_file:
        try:
            tweet = json.loads(line)
            tweets_data.append(tweet)
        except:
            continue

In [39]:
print(len(tweets_data))

83055


In [42]:
for i,item in enumerate(tweets_data[0:5]):
    print(i, '-'*50)
    print(item['text'])
    print(item['lang'])
    print(item['truncated'])

0 --------------------------------------------------
Grêmio escalado de forma oficial para a Semifinal da Copa do Brasil https://t.co/IfEec4EYUK
pt
False
1 --------------------------------------------------
Me inscrevi em 3 editais de treinamento profissional pela uf, me inscrevi em duas vagas de estágio pela net.
Confio… https://t.co/kqZEy0tJQE
pt
True
2 --------------------------------------------------
Grêmio definido para a semifinal da Copa do Brasil
pt
False
3 --------------------------------------------------
RT @lets_mariano: se a 2° temporada não sair logo eu vou levar a netflix pras ideia https://t.co/YboDVLVgUV
pt
False
4 --------------------------------------------------
RT @arctmankeys: ngm:

estudante de medicina no primeiro semestre: https://t.co/RuFT970Otc
pt
False


In [43]:
tweets = pd.DataFrame(columns=['text','lang','hashtags','user','location','truncated'])
tweets

,text,lang,hashtags,user,location,truncated


In [44]:
for x,item in enumerate(tweets_data):
    text = ""

    try:
        if item['truncated']:
            text=item['extended_tweet']['full_text']
        else:
            text=item['text']
        tweets.loc[x]=[text,item['lang'],item['entities'],item['user']['screen_name'],item['user']['location'],item['truncated']]
        if x % 5000 == 0:
            print("[{}] - TEXT - {}".format(x,text))
    except:
        pass

[0] - TEXT - Grêmio escalado de forma oficial para a Semifinal da Copa do Brasil https://t.co/IfEec4EYUK
[5000] - TEXT - RT @arctmankeys: ngm:

estudante de medicina no primeiro semestre: https://t.co/RuFT970Otc
[10000] - TEXT - RT @jfrmnt: Lista de comunistas atualizada:
The NY Times
The Economist
Le Mond
Alemanha 
ONU
Madonna
Veja
Reinaldo Azevedo
Papa Francisco
G…
[15000] - TEXT - @BADBYEMP3 to quase man alguém me segura pra não mudar a senha da netflix
[20000] - TEXT - RT @Pedro_Britto16: A caminhada é o seguinte a segunda temporada tem que ter mais episodio certo família e a liberdade do irmão badá tem qu…
[25000] - TEXT - RT @lancnetflix: "Nerve" e mais 35 filmes que retornarão à Netflix: https://t.co/UKnBft3dpK https://t.co/1q3rndVWHo
[30000] - TEXT - RT @lelispatricia: Brasil é o único país em que o  pobre, sem estudo ou emprego, defende os cortes na educação e não consegue reconhecer o…
[35000] - TEXT - Eu pedi e o @Gremio  atendeu!!! Agora a taça!!!!
[40000] - TEXT - RT @beca

In [45]:
len(tweets[tweets.lang == 'pt'])

83055

In [46]:
pt_tweets = tweets[tweets.lang == 'pt'].reset_index(drop=True)
pt_tweets.loc[19]['text']

'@mahoubitch O maior erro do nerd foi existir, infelizmente'

In [47]:
def remove_user(text):
    text = text.split()
    users = []
    for item in text:
        try:
            result = re.search('@(.*)', item)
            name = result.group(1)
            users.append("@"+name)
        except:
            pass
    text_filtered = [x for x in text if x not in users]
    text_filtered = " ".join(text_filtered)
    return text_filtered, users

In [48]:
def remove_latin_character(text):
    text = unicodedata.normalize('NFKD',text).encode('ASCII','ignore').decode("utf-8") # remove latin utf-8 characters
    return text

In [49]:
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

In [50]:
def remove_hashtag(text):
    text = text.split()
    hashtags = []
    for item in text:
        try:
            result = re.search('#(.*)', item)
            name = result.group(1)
            hashtags.append("#"+name)
        except:
            pass
    text_filtered = [x for x in text if x not in hashtags]
    text_filtered = " ".join(text_filtered)
    return text_filtered, hashtags

In [51]:
def remove_link(text):
    text = text.split()
    links = []
    for item in text:
        try:
            result = re.search('http(.*)', item)
            name = result.group(1)
            links.append(item)
        except:
            pass
    text_filtered = [x for x in text if x not in links]
    text_filtered = " ".join(text_filtered)
    return text_filtered, links

## Preprocessing Text

In [53]:
def preprocess(text):
    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = text.replace('\n', ' ')
    # Remove user
    text, _ = remove_user(text)
    # Remove hashtag
    text, _ = remove_hashtag(text)
    # Remove links 
    text, _ = remove_link(text)
    # Remove punctuation
    text = strip_punctuation(text)
    # Remove latin character
    text = remove_latin_character(text)
    # Removing non ASCII chars    
    text = re.sub(r'[^\x00-\x7f]',r' ',text)
    # Remove "RT" string
    text = text.replace('rt', ' ')
    #text = re.sub(r'[^\w\s]','',text) # strip punctuation
    text = ' '.join(text.split()) ## replacing extra spaces
    text = text.replace(".","")
    return text

In [54]:
pt_tweets['clean_text'] = pt_tweets['text'].apply(lambda x: preprocess(x))

In [55]:
pt_tweets.loc[19]['clean_text']

'o maior erro do nerd foi existir infelizmente'

In [56]:
for i in range(0,20):
    print(i,"-", pt_tweets['clean_text'][i])

0 - gremio escalado de forma oficial para a semifinal da copa do brasil
1 - me inscrevi em 3 editais de treinamento profissional pela uf me inscrevi em duas vagas de estagio pela net confio que nao vou passar em nenhum
2 - gremio definido para a semifinal da copa do brasil
3 - se a 2 temporada nao sair logo eu vou levar a netflix pras ideia
4 - ngm estudante de medicina no primeiro semestre
5 - ja estamos em casa para a primeira pa ida da semifinal da
6 - to queria um emprego p bancar minha manutencao na facul slc ir p sp todo dia me desanima mt daqui
7 - ouvi essa frase no trampo hj tem gente que quer emprego mas nao quer trabalhar so verdades kdfkkffk
8 - parabens gremio pela classificacao grenal na final
9 - nossa mas eu vou fazer tanto isso
10 - vou esperar a guitarra hehehe dalhe nosso gremio imo al
11 - ja estamos em casa para a primeira pa ida da semifinal da
12 - desisto de engenharia
13 - caralho que foda acabei de ver algumas paradas do novo especial da netflix do whindersson

## Training W2V

In [57]:
pt_tweets.tail(5)

,text,lang,hashtags,user,location,truncated,clean_text
83050,"RT @futebol_pais: Na estreia de Zé Ricardo, In...",pt,"{'hashtags': [], 'urls': [], 'user_mentions': ...",reidalliga12,None,False,na estreia de ze ricardo inter bateu o fo alez...
83051,RT @thmwrvel: faço parte do 1% do público da N...,pt,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",Ruan37175716,None,False,faco pa e do 1 do publico da netflix que nao t...
83052,RT @madureberti: a menina romântica q habita e...,pt,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",Barbosa9Marta,None,False,a menina romantica q habita em mim agradece
83053,Meu maxilar tá até doendo de tanto rir do show...,pt,"{'hashtags': [], 'urls': [], 'user_mentions': ...",whysosexydemi,journalism & photography,False,meu maxilar ta ate doendo de tanto rir do show...
83054,RT @madureberti: a menina romântica q habita e...,pt,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",a_furtado2,None,False,a menina romantica q habita em mim agradece


In [58]:
raw_sentences = list(pt_tweets['clean_text'])

In [59]:
sentences = []
for sent in raw_sentences:
    sentences.append(sent.split())

In [60]:
sentences[-1]

['a', 'menina', 'romantica', 'q', 'habita', 'em', 'mim', 'agradece']

In [61]:
bigram = Phrases(sentences,min_count=1)

## Training Hyperparams

In [62]:
embedded_size = 50
context_window = 8
workers=4
min_count = 1

In [63]:
model = gensim.models.Word2Vec(
        sentences = sentences,
        #sentences = bigram[sentences],
        size = embedded_size, 
        window = context_window,
        workers = workers,
        min_count = min_count,
        #sample = downsampling,
        #hs = 1,
        #negative = 0
)

In [64]:
model.wv.most_similar('gremio')

[('inter', 0.8490924835205078),
 ('athletico', 0.8117737770080566),
 ('time', 0.8012363314628601),
 ('gremista', 0.7775830030441284),
 ('jogo', 0.7613232135772705),
 ('flamengo', 0.7354684472084045),
 ('atletico', 0.7266033887863159),
 ('rival', 0.706829309463501),
 ('tardelli', 0.7044202089309692),
 ('cruzeiro', 0.6981300115585327)]

In [65]:
model.wv.most_similar('inter')

[('gremio', 0.8490924835205078),
 ('athletico', 0.7781550884246826),
 ('jogo', 0.776072084903717),
 ('flamengo', 0.7659093141555786),
 ('cruzeiro', 0.7543831467628479),
 ('time', 0.7479128241539001),
 ('palmeiras', 0.7107430696487427),
 ('reposicao', 0.6960253715515137),
 ('atletico', 0.6947750449180603),
 ('mineirao', 0.6785922646522522)]

In [66]:
model.wv.most_similar('politica')

[('ponta', 0.8588873147964478),
 ('pesquisas', 0.8373178243637085),
 ('deterioracao', 0.8326338529586792),
 ('investimento', 0.8305712938308716),
 ('pnd', 0.8268405199050903),
 ('servicos', 0.82682865858078),
 ('investimentos', 0.8263817429542542),
 ('eram', 0.8202065229415894),
 ('53', 0.8166924118995667),
 ('autentica', 0.8157235383987427)]

In [67]:
model.wv.most_similar('netflix')

[('sintonia', 0.6679555773735046),
 ('05x01', 0.6262835264205933),
 ('continuacao', 0.6036708354949951),
 ('terminei', 0.602993369102478),
 ('filme', 0.599368691444397),
 ('starz', 0.5984426140785217),
 ('seriados', 0.5753488540649414),
 ('saga', 0.5657902359962463),
 ('microondas', 0.5573458075523376),
 ('garotinha', 0.5448958873748779)]

In [68]:
model.wv.save_word2vec_format('vetores.bin', binary=True)

/Users/rsilvei/Envs/nlp_new/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [69]:
!ls -lah

total 14240
drwxr-xr-x  9 rsilvei  ADESI\Domain Users   288B Aug 20 14:22 .
drwxr-xr-x  6 rsilvei  ADESI\Domain Users   192B Aug 14 10:51 ..
drwxr-xr-x  3 rsilvei  ADESI\Domain Users    96B Aug 20 13:19 .ipynb_checkpoints
-rw-r--r--  1 rsilvei  ADESI\Domain Users    27K Aug 20 14:21 1.word2vec_trainer.ipynb
drwxr-xr-x  6 rsilvei  ADESI\Domain Users   192B Aug 20 13:18 demo_1
drwxr-xr-x  3 rsilvei  ADESI\Domain Users    96B Aug 14 18:31 demo_2
drwxr-xr-x  3 rsilvei  ADESI\Domain Users    96B Aug 14 18:30 demo_3
drwxr-xr-x  3 rsilvei  ADESI\Domain Users    96B Aug 14 18:34 demo_4
-rw-r--r--  1 rsilvei  ADESI\Domain Users   6.9M Aug 20 14:22 vetores.bin


In [75]:
model.wv.most_similar(positive=['gremio', 'inter'], negative=['cebolinha'])

[('2919', 0.6444302797317505),
 ('jogo', 0.6411250829696655),
 ('corretas', 0.6131941080093384),
 ('flamengo', 0.607387125492096),
 ('bank', 0.5846786499023438),
 ('grenal', 0.5808649659156799),
 ('time', 0.5799593329429626),
 ('santostemos', 0.5762511491775513),
 ('intergremio', 0.5759652256965637),
 ('aston', 0.5759439468383789)]